In [2]:
import pandas as pd
import sys
sys.path.append('/')
from utils_processing import transform_verbose_to_calendar
from utils_plot import plot_bar_time_series, create_map, create_tree

In [3]:
df_verbose = pd.read_csv('../data/trips_verbose.csv')
df = transform_verbose_to_calendar(df_verbose, save=False)
df = df[df['country'] != 'All']

plot_bar_time_series(
    df, 
    select_type='travel', 
    time_period='year', 
    title='Countries Been', 
    #save_path='../attachments/bar_time_series.html',
    save_path=None
    )

In [7]:
COUNTRIES_LIVED = ['China','United States','United Kingdom','Sweden']
COLOR_DICT = {'Asia':'gold', 'Europe':'royalblue', 'America':'purple', 'Africa':'crimson'}

create_tree(
    df,
    feat='country',
    var='total_days',
    flag='continent',
    color_dict=COLOR_DICT,
    threshold=365,
    threshold_global=True,
    height=None,
    #save_path='../attachments/tree_travel_all.html',
    save_path=None
    )

In [12]:
create_tree(
    df[~df.country.isin(COUNTRIES_LIVED)],
    feat='country',
    var='total_days',
    flag='continent',
    color_dict=COLOR_DICT,
    threshold=3,
    threshold_global=False,
    height=None,
    #save_path='../attachments/tree_travel_visits.html',
    save_path=None
    )

In [5]:
VAR = 'total_days'

create_map(
    df, 
    var=VAR, 
    code_convention='code3', 
    bins=[0, 7, 30, 365, df[VAR].max()],
    color='royalblue',
    projection_type="orthographic", # orthographic, equirectangular, mercator, natural earth, kavrayskiy7, miller, robinson, eckert4, azimuthal equal area, azimuthal equidistant, gnomonic, stereographic, mollweide
    tooltip_mode='calendar',
    save_path='../attachments/map_travel.html'
    #save_path=None
    )